In [6]:
import pandas as pd
import math
import pandas as pd
import numpy as np
import requests
import os
from bs4 import BeautifulSoup
import json

In [7]:
## Defining Function to Convert Lat-Long into xy Coordinates 
## Web Mercator Auxiliary Sphere tiling scheme (WKID 102100)

def geographic_to_web_mercator_x(x_lon):
    if abs(x_lon) <= 180:
        num = x_lon * 0.017453292519943295
        x = 6378137.0 * num
        x_mercator = x
        return x_mercator
    else:
        print('Invalid coordinate values for conversion')
        
def geographic_to_web_mercator_y(y_lat):
    if abs(y_lat) < 90:
        a = y_lat * 0.017453292519943295
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))
        return y_mercator
    else:
        print('Invalid coordinate values for conversion')

        
print(geographic_to_web_mercator_x(35.932778),geographic_to_web_mercator_y(31.949722))


4000018.549747743 3756712.6662826063


In [9]:
## Substitute to main table if needed 

df=pd.read_csv('worldcities.csv',encoding='utf-8')
coord = df[['city', 'lat','lng']]
coord.info()

## Input lat-long coordinates for conversion

lat = df['lat']
long = df['lng']
x = []
y = []

for i in long:
    x.append(geographic_to_web_mercator_x(i))
    
for j in lat:
    y.append(geographic_to_web_mercator_y(j))
    
## Create x,y coordinate columns 
    
coord['x'] = x
coord['y'] = y

## Split to smaller chunks 

size = 1000
coord_split = coord.iloc[:size, :]
coord_split.info()


coord_test = coord.iloc[:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42905 entries, 0 to 42904
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    42905 non-null  object 
 1   lat     42905 non-null  float64
 2   lng     42905 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1005.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    1000 non-null   object 
 1   lat     1000 non-null   float64
 2   lng     1000 non-null   float64
 3   x       1000 non-null   float64
 4   y       1000 non-null   float64
dtypes: float64(4), object(1)
memory usage: 39.2+ KB


<ipython-input-9-96acc3a53ff5>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord['x'] = x
<ipython-input-9-96acc3a53ff5>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord['y'] = y


In [10]:
## Landform-related attributes 

URL = []
bioclimate = []
landform = []
landcover = []

## Creating domain, fixed parameters for api call

domain = 'https://rmgsc.cr.usgs.gov/ArcGIS/rest/services/globalelus/MapServer/identify'
imageDisplay = [1147,61,96]
mapExtent= [-43330894.37039283,4832699.33817045,1557820.6084612794,7219980.6055724565]
param = {'f':'json',
            'returnFieldName': 'false',
            'returnGeometry' : 'false',
            'returnUnformattedValues': 'false',
            'returnZ' : 'false',
            'tolerance': 3
}

domain += '?'+'&'.join([k+'='+str(v) for (k,v) in param.items()])
domain += '&imageDisplay='+','.join(map(str, imageDisplay))
param = {'geometryType':'esriGeometryPoint',
         'sr':102100
        }
domain += '&'.join([k+'='+str(v) for (k,v) in param.items()])
domain += '&mapExtent='+','.join(map(str, mapExtent))
domain += '&layers=all:4'

## Retreive GIS text data by cities via x,y coordinates

for i in range(0,size):
    
    x_coor = x[i]
    y_coor = y[i]
    geometry2 = [{'x': x_coor},{'y':y_coor}]
    
    ## Creating and saving (optional) API call url by city
    api_call = domain + '&geometry={'+','.join(['%22'+k+'%22:'+str(v) for l in geometry2 for (k,v) in l.items() ])+'}'
    ## optional: URL.append(api_call)
    
    ## Slice json result from API GET request 
    response = requests.get(api_call)
    raw_data = response.json()
    total_results = raw_data['results'][0]
    data = total_results['attributes']
    raw_table = pd.DataFrame(data, index=[0])
    
    ## Retrieve target attributes 
    if 'EF_Bio_Des' in list(raw_table.columns.values): 
        bio = raw_table['EF_Bio_Des'].item()
        bioclimate.append(bio)   
    else:
        bioclimate.append('NA')
            
    if 'EF_LF_Desc' in list(raw_table.columns.values):
        lf = raw_table['EF_LF_Desc'].item()
        landform.append(lf) 
    else:
        landform.append('NA')

    if 'EF_GLC_Des' in list(raw_table.columns.values):
        lc = raw_table['EF_GLC_Des'].item()
        landcover.append(lc)  
    else:
        landcover.append('NA')

coord_split['Bioclimate'] = bioclimate
coord_split['Landform'] = landform
coord_split['Landcover'] = landcover
coord_split.to_csv('Landform Data.csv',index=False)


<ipython-input-10-489a13986702>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord_split['Bioclimate'] = bioclimate
<ipython-input-10-489a13986702>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coord_split['Landform'] = landform
<ipython-input-10-489a13986702>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [11]:
coord_split.head()

,city,lat,lng,x,y,Bioclimate,Landform,Landcover
0,Tokyo,35.6839,139.7744,1.555962e+07,4.257213e+06,Warm Very Wet,Flat or Nearly Flat Plains,Artificial surface or urban area
1,Jakarta,-6.2146,106.8451,1.189394e+07,-6.931666e+05,Very Hot Wet,Flat or Nearly Flat Plains,Artificial surface or urban area
2,Delhi,28.6667,77.2167,8.595724e+06,3.333292e+06,Very Hot Semi-Dry,Flat or Nearly Flat Plains,Artificial surface or urban area
3,Manila,14.6000,120.9833,1.346780e+07,1.643144e+06,Very Hot Very Wet,Flat or Nearly Flat Plains,Artificial surface or urban area
4,São Paulo,-23.5504,-46.6339,-5.191262e+06,-2.698717e+06,Hot Wet,Moderate Hills,Artificial surface or urban area
